In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
from utils.PredictSalesUtils import getTestEnriched 
from utils.PredictSalesUtils import generateFeaturesForEvaluation

# Test Data Loading 

In [3]:
test_df_enriched = getTestEnriched('../datasets/predict-sales/test.csv','../datasets/predict-sales/test.csv')

In [4]:
test_df_enriched.head()

,ID,shop_id,item_id,item_category_id,ID_pair,ID_CAT_pair
0,0,5,5037,19,5-5037,5-19
1,1,5,5320,55,5-5320,5-55
2,2,5,5233,19,5-5233,5-19
3,3,5,5232,23,5-5232,5-23
4,4,5,5268,20,5-5268,5-20


# Sales Direct Load

In [5]:
sales_df = pd.read_csv('../datasets/predict-sales/sales_train_enriched.csv')

In [6]:
sales_df.drop(labels=['Unnamed: 0'], inplace=True, axis=1)

In [7]:
sales_df.head()

,index,date_block_num,shop_id,item_id,item_price,item_cnt_day,ID_pair,item_category_id,ID_CAT_pair
0,0,0,59,22154,999.0,1.0,59-22154,37,59-37
1,3270,0,24,22154,999.0,1.0,24-22154,37,24-37
2,17081,0,27,22154,999.0,1.0,27-22154,37,27-37
3,25918,0,25,22154,999.0,1.0,25-22154,37,25-37
4,25919,0,25,22154,999.0,1.0,25-22154,37,25-37


# Evaluation Processing 

## Ensemble Process

In [8]:
slots = sales_df['date_block_num'].nunique()
slot_seq = np.arange(1,slots)

In [9]:
windows = [3,6,12,18,25,32]

In [10]:
def processFold(i):
    months = slot_seq[-windows[i-1]:]
    fold_df = generateFeaturesForEvaluation(sales_df, months, test_df_enriched)
    
    #sns.heatmap(~fold_df.isna())
    # TODO cleaning and imputing data
    
    # TODO Prediction in model fold[index]
        
    return 

### Fold 1

In [ ]:
result1 = processFold(1)

### Fold 2

In [ ]:
result2 = processFold(2)

### Fold 3

In [ ]:
result3 = processFold(3)

### Fold 4

In [ ]:
result4 = processFold(4)

### Fold 5

In [ ]:
result5 = processFold(5)

### Fold 6

In [ ]:
result6 = processFold(6)

## Stacking Process 

In [ ]:
# TODO

# End of Case! 